In [1]:
# Cell 1: Importations nécessaires
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration pour l'affichage de matplotlib
%matplotlib inline
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Cell 2: Configuration des chemins et chargement des données
# Assurez-vous que la variable d'environnement AIRFLOW_HOME est définie
AIRFLOW_HOME = os.environ.get('AIRFLOW_HOME')
if not AIRFLOW_HOME:
    raise EnvironmentError("La variable d'environnement AIRFLOW_HOME n'est pas définie. Veuillez la définir avant d'exécuter ce notebook.")

PROCESSED_DATA_PATH = os.path.join(AIRFLOW_HOME, 'data', 'processed')
MODELED_DATA_FILENAME = "modeled_weather_data.parquet"
MODELED_DATA_FILEPATH = os.path.join(PROCESSED_DATA_PATH, MODELED_DATA_FILENAME)

print(f"Tentative de chargement des données depuis : {MODELED_DATA_FILEPATH}")

try:
    df_modeled = pd.read_parquet(MODELED_DATA_FILEPATH)
    print("Données modélisées chargées avec succès.")
    print(f"Taille du DataFrame : {df_modeled.shape}")
    print("\nAperçu des 5 premières lignes :")
    print(df_modeled.head())
except FileNotFoundError:
    print(f"Erreur : Le fichier {MODELED_DATA_FILEPATH} n'a pas été trouvé.")
    print("Veuillez vous assurer d'avoir exécuté les scripts ETL et de modélisation (extract_data.py, transform_data.py, data_modeling.py) en premier.")
    df_modeled = pd.DataFrame() # DataFrame vide pour éviter les erreurs plus tard
except Exception as e:
    print(f"Une erreur inattendue est survenue lors du chargement des données : {e}")
    df_modeled = pd.DataFrame()


# Cell 3: Informations sur le DataFrame
if not df_modeled.empty:
    print("\nInformations sur le DataFrame (types de données, valeurs manquantes) :")
    df_modeled.info()

    print("\nStatistiques descriptives pour les colonnes numériques :")
    print(df_modeled.describe())

    print(f"\nNombre de villes uniques : {df_modeled['city'].nunique()}")
    print(f"Années présentes : {df_modeled['year'].unique()}")
    print(f"Mois présents : {df_modeled['month_name'].unique()}")
else:
    print("DataFrame vide, impossible d'afficher les informations.")

# Cell 4: Visualisation 1 - Température moyenne par mois pour une ville spécifique
if not df_modeled.empty:
    # Sélectionnez une ville pour l'exemple, ou demandez à l'utilisateur
    example_city = 'London'
    if example_city not in df_modeled['city'].unique():
        print(f"La ville '{example_city}' n'est pas présente dans les données. Choisissons la première ville disponible.")
        example_city = df_modeled['city'].iloc[0] if not df_modeled['city'].empty else None

    if example_city:
        df_city = df_modeled[df_modeled['city'] == example_city].sort_values(by=['year', 'month'])

        plt.figure(figsize=(12, 6))
        sns.lineplot(data=df_city, x='month_name', y='avg_temp_celsius', hue='year', marker='o')
        plt.title(f'Température Moyenne Mensuelle pour {example_city} par Année')
        plt.xlabel('Mois')
        plt.ylabel('Température Moyenne (°C)')
        plt.xticks(rotation=45)
        plt.legend(title='Année')
        plt.tight_layout()
        plt.show()
    else:
        print("Aucune ville disponible pour la visualisation.")
else:
    print("DataFrame vide, impossible de créer la visualisation 1.")

# Cell 5: Visualisation 2 - Précipitations totales par mois pour quelques villes
if not df_modeled.empty:
    # Sélectionnez quelques villes pour la comparaison
    comparison_cities = ['New York', 'Tokyo', 'Paris']

    # Filtrez les villes qui existent réellement dans le DataFrame
    available_comparison_cities = [city for city in comparison_cities if city in df_modeled['city'].unique()]

    if not available_comparison_cities:
        print("Aucune des villes de comparaison spécifiées n'est présente dans les données. Choisissons quelques villes aléatoires.")
        available_comparison_cities = df_modeled['city'].sample(min(3, df_modeled['city'].nunique())).tolist()

    df_comp = df_modeled[df_modeled['city'].isin(available_comparison_cities)].sort_values(by=['city', 'year', 'month'])

    plt.figure(figsize=(14, 7))
    sns.barplot(data=df_comp, x='month_name', y='total_precipitation_mm', hue='city', ci=None, estimator=sum) # Sum pour total par mois
    plt.title('Précipitations Totales Mensuelles par Ville')
    plt.xlabel('Mois')
    plt.ylabel('Précipitations Totales (mm)')
    plt.xticks(rotation=45)
    plt.legend(title='Ville')
    plt.tight_layout()
    plt.show()
else:
    print("DataFrame vide, impossible de créer la visualisation 2.")

# Cell 6: Visualisation 3 - Répartition de la température (histogramme)
if not df_modeled.empty:
    plt.figure(figsize=(10, 6))
    sns.histplot(df_modeled['avg_temp_celsius'], bins=20, kde=True)
    plt.title('Distribution de la Température Moyenne Mensuelle')
    plt.xlabel('Température Moyenne (°C)')
    plt.ylabel('Fréquence')
    plt.tight_layout()
    plt.show()
else:
    print("DataFrame vide, impossible de créer la visualisation 3.")

# Cell 7: Tableaux d'agrégation supplémentaires (ex: top 5 des mois les plus pluvieux)
if not df_modeled.empty:
    print("\nTop 5 des mois les plus pluvieux (toutes villes confondues) :")
    top_rainy_months = df_modeled.groupby(['year', 'month_name']).agg(
        total_global_precipitation=('total_precipitation_mm', 'sum')
    ).sort_values(by='total_global_precipitation', ascending=False).head(5)
    print(top_rainy_months)

    print("\nTop 5 des villes les plus chaudes (moyenne sur toutes les données) :")
    top_hot_cities = df_modeled.groupby('city').agg(
        overall_avg_temp=('avg_temp_celsius', 'mean')
    ).sort_values(by='overall_avg_temp', ascending=False).head(5)
    print(top_hot_cities)
else:
    print("DataFrame vide, impossible de créer les tableaux d'agrégation.")

OSError: La variable d'environnement AIRFLOW_HOME n'est pas définie. Veuillez la définir avant d'exécuter ce notebook.